# Tools and Tool Calling with LLMs in LangChain
* Notebook by Adam Lang
* Date: 9/27/2024

# Overview
* In this notebook we will implement tools and the process of tool calling using LangChain.

## Install dependencies

In [1]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 974.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Search APIs Installation
* We will use search APIs as tools:
1. duckduckgo
2. wikipedia
3. rich

In [2]:
## install search apis
!pip install duckduckgo-search
!pip install wikipedia
## to highlight json
!pip install rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 35.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=67b0d8c5e5029fbf45404cf20b2bf1d7fd5bbc746ee335eb054aefde3beb74ac
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


## Enter Open AI API Key

In [3]:
from getpass import getpass

OPENAI_KEY = getpass('Enter your Open AI API key: ')

Enter your Open AI API key: ··········


## Enter Tavily Search API Key
* https://tavily.com/

In [4]:
TAVILY_API_KEY = getpass('Enter your Tavily Search API Key: ')

Enter your Tavily Search API Key: ··········


## Set Environment Variables

In [5]:
## env variables
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

## Wikipedia Tool
* This tool allows you to utilize the Wikipedia open source API to search wikipedia for information.

In [6]:
## imports
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

## instantiate wrapper --> get top 2 results (top k)
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=8000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper, features='lxml')


In [7]:
## get description
wiki_tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [8]:
## wiki args
wiki_tool.args

{'query': {'title': 'Query', 'type': 'string'}}

In [13]:
## query using wikipedia tool
print(wiki_tool.invoke({'query': "Vail Resorts"}))

Page: Vail Resorts
Summary: Vail Resorts, Inc. is an American mountain resort company headquartered in Broomfield, Colorado. The company is divided into three divisions. The mountain segment owns and operates 42 mountain resorts in four countries. Vail Resorts Hospitality owns or manages hotels, lodging, condominiums, and golf courses, and the Vail Resorts Development Company oversees property development and real estate holdings.



Page: Vail Ski Resort
Summary: Vail Ski Resort is a ski resort in the western United States, located near the town of Vail in Eagle County, Colorado. At 5,289 acres (8.3 sq mi; 21.4 km2), it is the third-largest single-mountain ski resort in the U.S., behind Big Sky and Park City, featuring seven bowls and intermediate gladed terrain in Blue Sky Basin.
Opened in late 1962, Vail is one of 37 mountain resorts owned and operated by Vail Resorts, which also operates three other nearby ski resorts (Beaver Creek, Breckenridge, and Keystone).
Vail Mountain has th

Summary:
* We can see it pulled up the top 2 results for my query of "Vail Resorts".

In [14]:
## query again
print(wiki_tool.invoke({'query': 'AI'}))

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Artificial intelligence
Summary: Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
Some high-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); interacting via human speech (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT, and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI becaus

Customizing default tool with own name, description, and other details as we see below:

In [15]:
from langchain.agents import Tool

## customize wikipedia tool --> tool always nees a function (func)
wiki_tool_init = Tool(name='Wikipedia',
                      func=wiki_api_wrapper.run,
                      description='useful when you need a detailed answer about encyclopedic general knowledge.')

In [16]:
## get description of custom tool
wiki_tool_init.description

'useful when you need a detailed answer about encyclopedic general knowledge.'

In [17]:
## get wiki tool args
wiki_tool_init.args

{'tool_input': {'type': 'string'}}

In [18]:
## invoke custom wiki tool with query
print(wiki_tool_init.invoke({'tool_input': 'diversity, equity, inclusion'}))

Page: Diversity, equity, and inclusion
Summary: Diversity, equity, and inclusion (DEI) are organizational frameworks which seek to promote the fair treatment and full participation of all people, particularly groups who have historically been underrepresented or subject to discrimination on the basis of identity or disability. These three notions (diversity, equity, and inclusion) together represent "three closely linked values" which organizations seek to institutionalize through DEI frameworks. The concepts predate this terminology and other variations include diversity, equity, inclusion and belonging (DEIB), inclusion and diversity (I&D), justice, equity, diversity and inclusion (JEDI or EDIJ), or diversity, equity, inclusion and accessibility (IDEA, DEIA or DEAI).
Diversity refers to the presence of variety within the organizational workforce, such as in identity and identity politics. It includes gender, ethnicity, sexual orientation, disability, age, culture, class, religion, or

## Tavily Search Tool
* The Tavily Search API is a search engine optimized for LLMs and RAG, aimed at efficient, quick and persistent search results.

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

##setup tavily tool
tavily_tool = TavilySearchResults(max_results=2)

In [20]:
## tavily args
tavily_tool.args

{'query': {'title': 'Query',
  'description': 'search query to look up',
  'type': 'string'}}

In [21]:
## tavily description
tavily_tool.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [23]:
## get results
results = tavily_tool.invoke("Tell me about Vermont")
results

[{'url': 'https://www.allamericanatlas.com/vermont-fun-facts/',
  'content': '3. The state\'s capital is Montpelier, and its largest city is Burlington. 4. Vermont is known as the "Green Mountain State.". 5. The state is famous for its picturesque landscapes, including its rolling green hills and stunning fall foliage. 7. Vermont is the leading producer of maple syrup in the United States. 8.'},
 {'url': 'https://www.britannica.com/place/Vermont',
  'content': 'Vermont has never stood in the mainstream of the country’s history, but its people and land have poured into their country a strength and a sense of continuity that joins the achievements of the nation’s past with the purposes of its present. The steeples of white wooden churches rising above mountain-bound small towns with trim village greens, the herds of dairy cattle on sloping mountain pastures, and the red-gold leaves of tree-lined autumnal lanes are aspects of scenic Vermont that, in painting and photography, have become s

In [28]:
## get results
results = tavily_tool.invoke("What is the current weather in Amsterdam? from weatherapi")
results

[{'url': 'https://www.omakchronicle.com/news/national/dutch-hit-uber-with-huge-fine-over-driver-data/article_6d686ed2-8fca-59e9-9b56-c8d1e8e5d6a5.html',
  'content': 'Sunshine and clouds mixed. High near 85F. Winds SSW at 10 to 15 mph..\r\n                                \nCloudy. Low 61F. Winds SSW at 5 to 10 mph.\n Updated: August 26, 2024 @ 3:36 am\n\n\n\nDutch hit Uber'},
 {'url': 'https://www.finextra.com/newsarticle/43975/german-bank-lbbw-enters-crypto-market',
  'content': "Write a blog post about this story (membership required) Trending 10 Apr 3 4 09 Apr 0 2 2 08 Apr 2 5 09 Apr 0 1 1 Bitpanda crypto exchange spins out with €30m funding 29 June 2023 0 3 3 Bitpanda to invest $10 million in AI 16 May 2023 0 3 Crypto exchange Bitpanda makes job cuts 24 Jun 2022 JPMorgan's Barraclough joins Bitpanda as CEO of digital assets exchange 28 03 Aug 2021 Bitpanda to create 300 jobs at Polish tech hub 17 Dec 2020 Definitive Differentiators - Forging a future-proof payments model 349\xa0dow

In [29]:
## get content
results[0]['content']

'Sunshine and clouds mixed. High near 85F. Winds SSW at 10 to 15 mph..\r\n                                \nCloudy. Low 61F. Winds SSW at 5 to 10 mph.\n Updated: August 26, 2024 @ 3:36 am\n\n\n\nDutch hit Uber'

In [31]:
## another example
results = tavily_tool.invoke("Get current weather in Zurich, from weatherapi")
results

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Zurich', 'region': '', 'country': 'Switzerland', 'lat': 47.37, 'lon': 8.55, 'tz_id': 'Europe/Zurich', 'localtime_epoch': 1727483240, 'localtime': '2024-09-28 02:27'}, 'current': {'last_updated_epoch': 1727482500, 'last_updated': '2024-09-28 02:15', 'temp_c': 11.4, 'temp_f': 52.5, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 235, 'wind_dir': 'SW', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 76, 'cloud': 25, 'feelslike_c': 10.6, 'feelslike_f': 51.1, 'windchill_c': 6.8, 'windchill_f': 44.2, 'heatindex_c': 8.2, 'heatindex_f': 46.7, 'dewpoint_c': 5.1, 'dewpoint_f': 41.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 9.8, 'gust_kph': 15.8}}"},
 {'url': 'https://www.meteosource.com/weather-api-zurich',
  'content': 'Weather API -

In [32]:
import json
import rich

## rich formats json
rich.print_json(
    results[0]['content'].replace('\'','\"')
)

{
  "location": {
    "name": "Zurich",
    "region": "",
    "country": "Switzerland",
    "lat": 47.37,
    "lon": 8.55,
    "tz_id": "Europe/Zurich",
    "localtime_epoch": 1727483240,
    "localtime": "2024-09-28 02:27"
  },
  "current": {
    "last_updated_epoch": 1727482500,
    "last_updated": "2024-09-28 02:15",
    "temp_c": 11.4,
    "temp_f": 52.5,
    "is_day": 0,
    "condition": {
      "text": "Partly cloudy",
      "icon": "//cdn.weatherapi.com/weather/64x64/night/116.png",
      "code": 1003
    },
    "wind_mph": 5.1,
    "wind_kph": 8.3,
    "wind_degree": 235,
    "wind_dir": "SW",
    "pressure_mb": 1012.0,
    "pressure_in": 29.88,
    "precip_mm": 0.0,
    "precip_in": 0.0,
    "humidity": 76,
    "cloud": 25,
    "feelslike_c": 10.6,
    "feelslike_f": 51.1,
    "windchill_c": 6.8,
    "windchill_f": 44.2,
    "heatindex_c": 8.2,
    "heatindex_f": 46.7,
    "dewpoint_c": 5.1,
    "dewpoint_f": 41.1,
    "vis_km": 10.0,
    "vis_miles": 6.0,
    "uv": 1.0,
    "gust_mph": 9.8,
    "gust_kph": 15.8
  }
}

## DuckDuckGo Search Tool
* Search tool for words, documents, images, videos, news, maps and text translation using the DuckDuckGo.com search engine.

In [33]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

# we choose wt-wt to just do generic and not region specific search
# go to https://github.com/deedy5/duckduckgo_search?tab=readme-ov-file#regions for more details
wrapper = DuckDuckGoSearchAPIWrapper(region='wt-wt', max_results=2)
ddgs_tool = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')

In [34]:
## get results
results = ddgs_tool.invoke('Tell me about New York City')
results

"[snippet: New York City, city and port located at the mouth of the Hudson River, southeastern New York state, northeastern U.S. It is the largest and most influential American metropolis, encompassing Manhattan and Staten islands, the western sections of Long Island, and a small portion of the New York state mainland to the north of Manhattan. New York City is in reality a collection of many ..., title: New York City | Layout, Map, Economy, Culture, Facts, & History, link: https://www.britannica.com/place/New-York-City], [snippet: This 5-day New York City itinerary is all about seeing the top sights and getting to know the food, history, and culture of this iconic city. We've listed some of the top attractions, like visiting the 9/11 Memorial or the Empire State Building, and also some fun local New Yorker activities!You can visit Chelsea Market for some amazing food, hit a speakeasy, or hang out in Central Park ..., title: How to Spend 5 Days in NYC: A Complete New York Itinerary, li

In [35]:
## clean up result
results = [doc.strip('[]') for doc in results.split('], [')]
results

['snippet: New York City, city and port located at the mouth of the Hudson River, southeastern New York state, northeastern U.S. It is the largest and most influential American metropolis, encompassing Manhattan and Staten islands, the western sections of Long Island, and a small portion of the New York state mainland to the north of Manhattan. New York City is in reality a collection of many ..., title: New York City | Layout, Map, Economy, Culture, Facts, & History, link: https://www.britannica.com/place/New-York-City',
 "snippet: This 5-day New York City itinerary is all about seeing the top sights and getting to know the food, history, and culture of this iconic city. We've listed some of the top attractions, like visiting the 9/11 Memorial or the Empire State Building, and also some fun local New Yorker activities!You can visit Chelsea Market for some amazing food, hit a speakeasy, or hang out in Central Park ..., title: How to Spend 5 Days in NYC: A Complete New York Itinerary, l

In [36]:
## another example with weather
results = ddgs_tool.invoke('What is the current weather in Zurich')
results

"[snippet: Allergy Outlook. Zurich, Zurich, Switzerland Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days., title: Zurich, Zurich, Switzerland Weather Forecast | AccuWeather, link: https://www.accuweather.com/en/ch/zurich/8050/weather-forecast/406097_pc], [snippet: Everything you need to know about today's weather in Zurich, Zurich, Switzerland. High/Low, Precipitation Chances, Sunrise/Sunset, and today's Temperature History., title: Weather Today for Zurich, Zurich, Switzerland | AccuWeather, link: https://www.accuweather.com/en/ch/zurich/8000/weather-today/405974_pc], [snippet: Zrich Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Zrich area., title: Zürich, Switzerland Weather Conditions | Weather Underground, link: https://www.wunderground.com/weather/ch/zürich], [snippet: Temperatures peaking at 64 °F. Overnight into Tuesday blows a li

In [37]:
## results again
results = [doc.strip('[]') for doc in results.split('], [')]
results

['snippet: Allergy Outlook. Zurich, Zurich, Switzerland Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days., title: Zurich, Zurich, Switzerland Weather Forecast | AccuWeather, link: https://www.accuweather.com/en/ch/zurich/8050/weather-forecast/406097_pc',
 "snippet: Everything you need to know about today's weather in Zurich, Zurich, Switzerland. High/Low, Precipitation Chances, Sunrise/Sunset, and today's Temperature History., title: Weather Today for Zurich, Zurich, Switzerland | AccuWeather, link: https://www.accuweather.com/en/ch/zurich/8000/weather-today/405974_pc",
 'snippet: Zrich Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the Zrich area., title: Zürich, Switzerland Weather Conditions | Weather Underground, link: https://www.wunderground.com/weather/ch/zürich',
 'snippet: Temperatures peaking at 64 °F. Overnight into Tuesday blows a

# Build your own tools in LangChain
* Tools are interfaces that an agent, chain, or LLM can use to interact with the world. They combine a few things:
* The name of the tool
* A description of what the tool is
* JSON schema of what the inputs to the tool are
* The function to call
* Whether the result of a tool should be returned directly to the user

It is useful to have this info because it can be useful to build action-taking systems.
  * The name, description, and JSON schema can be used to prompt the LLM so it knows how to specify what action to take and then the function to call is equivalent to taking that action.

Building a very simple tool which does basic math.

In [41]:
## math tool
from langchain_core.tools import tool

@tool
def divide(a, b):
  """Divide two numbers."""
  return a // b

# inspect some attributes associated with the tool
print(divide.name)
print(divide.description)
print(divide.args)

divide
Divide two numbers.
{'a': {'title': 'A'}, 'b': {'title': 'B'}}


In [42]:
type(divide)

langchain_core.tools.structured.StructuredTool

In [43]:
## invoke
divide.invoke({'a': 10, 'b': 2})

5

In [44]:
## invoke
divide.invoke({'a':10.5, 'b': 2.5})

4.0

Let's now build a tool with data type enforcing

In [45]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import StructuredTool

## create pydantic class
class CalculatorInput(BaseModel):
  a: float = Field(description="first number")
  b: float = Field(description="second number")

# divide function
def divide(a: float, b: float) -> float:
  """Divide two numbers."""
  return a // b

# we could use @tool decorator from above
calculator = StructuredTool.from_function(
    func=divide,
    name='Calculator',
    description='use to divide numbers',
    args_schema=CalculatorInput,
    return_direct=True
)


## lets see attributes of the tool
print(calculator.name)
print(calculator.description)
print(calculator.args)

Calculator
use to divide numbers
{'a': {'title': 'A', 'description': 'first number', 'type': 'number'}, 'b': {'title': 'B', 'description': 'second number', 'type': 'number'}}


In [46]:
## invoke
calculator.invoke({'a': 10, 'b': 2})

5.0

In [ ]:
## this errors becuase its not floating point num
calculator.invoke({'a': 2, 'b': 'abc'})

# LLM Tool Calling in LangChain
* Given a list of toools and a prompt, an LLM can request that 1 or more tools be invoked with appropriate args.
* Tool calling often known as "function calling" is the process of LLM identifying which tools or functions to call based on input prompts.

* Closed source LLMs such as GPT and Claude have tools baked into their infrastructure, whereas open-source LLMs often do not.
* We will look at both types.

## LLM tool calling with custom tools


In [49]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
import rich

## custom tool #1
@tool
def subtract(a: float, b: float) -> float:
  """Subtracts a and b."""
  print(a - b)

## custom tool #2
@tool
def multiply(a: float, b: float) -> float:
  """Multiplies a and b."""
  print(a * b)

## custom tool #3
@tool
def search_web(query: str) -> list:
  """Search the web for a query."""
  tavily_tool = TavilySearchResults(max_results=2)
  results = tavily_tool.invoke(query)
  for result in results:
    print(result['content'])

## custom tool #4
@tool
def get_weather(query: str) -> list:
  """Search weatherapi to get current weather."""
  if 'weather' not in query: #guides search tool to use weatherapi
    query = query + " weather from weatherapi"
  tavily_tool = TavilySearchResults(max_results=1)
  result = tavily_tool.invoke(query)
  rich.print_json(result[0]['content'].replace('\'', '\"'))

## custom tools
tools = [subtract, multiply, search_web, get_weather]

## Tool calling with LLMs with native support for tool or function calling
* Many LLM providers such as Anthropic, Cohere, Google, Mistral, OpenAI, and others, support variants of tool calling features.
* These features typically allow requests to the LLM to include available tools and their schemas and for responses to include calls to the tools.

In [68]:
## openai example
from langchain_openai import ChatOpenAI

## llm
chatgpt = ChatOpenAI(model='gpt-4', temperature=0)

In [69]:
## call bind_tools function
chatgpt_with_tools = chatgpt.bind_tools(tools)

In [88]:
## prompt
prompt = """
            Given the tools at your disposal, mention which tools to call for the following tasks:
            1. What is 2.1 times 3.5
            2. What is 30 - 12
            3. What is the current weather in Zurich today
            4. Can you tell me about Greenland and its capital
        """

## invoke model and get results
results = chatgpt_with_tools.invoke(prompt)

In [89]:
results

AIMessage(content='1. For the task "What is 2.1 times 3.5", you would call the `functions.multiply` tool.\n2. For the task "What is 30 - 12", you would call the `functions.subtract` tool.\n3. For the task "What is the current weather in Zurich today", you would call the `functions.get_weather` tool.\n4. For the task "Can you tell me about Greenland and its capital", you would call the `functions.search_web` tool.', response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 195, 'total_tokens': 301, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9bd56526-b2e8-4bc8-b72c-387587dfdca5-0')

In [90]:
results.tool_calls

[]

You have to call the tools yourself, the LLM wont do it for you

In [91]:
## toolkit defined
toolkit = {
    "subtract": subtract,
    "multiply": multiply,
    "search_web": search_web,
    "get_weather": get_weather
}

for tool_call in results.tool_calls:
  selected_tool = toolkit[tool_call['name'].lower()]
  print(f"Calling tool: {tool_call['name']}")
  tool_output = selected_tool.invoke(tool_call['args'])
  print()

In [92]:
tools

[StructuredTool(name='subtract', description='Subtracts a and b.', args_schema=<class 'pydantic.v1.main.subtractSchema'>, func=<function subtract at 0x7ac9794fd6c0>),
 StructuredTool(name='multiply', description='Multiplies a and b.', args_schema=<class 'pydantic.v1.main.multiplySchema'>, func=<function multiply at 0x7ac9794a8ca0>),
 StructuredTool(name='search_web', description='Search the web for a query.', args_schema=<class 'pydantic.v1.main.search_webSchema'>, func=<function search_web at 0x7ac9794a8d30>),
 StructuredTool(name='get_weather', description='Search weatherapi to get current weather.', args_schema=<class 'pydantic.v1.main.get_weatherSchema'>, func=<function get_weather at 0x7ac9794a8c10>)]

## Tool Calling for LLMs without native support for tool or function calling

Some models like ChatGPT are fine tuned for tool calling and provide a dedicated APIfor tool calling. Generally speaking such models are better at tool calling than non-fine tuned models.

Here we will explore an alternative method to invoke tools if youre using a model that does not natively support tool calling (even though we are using ChatGPT here which supports it, we will assume it can be any LLM which doesn't support tool calling).

To do this we write a prompt that will get the model to invoke the appropriate tools.

In [93]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

## rendered tools
rendered_tools = render_text_description(tools)
print(rendered_tools)

subtract(a: float, b: float) -> float - Subtracts a and b.
multiply(a: float, b: float) -> float - Multiplies a and b.
search_web(query: str) -> list - Search the web for a query.
get_weather(query: str) -> list - Search weatherapi to get current weather.


In [94]:
## here is the prompt we need
system_prompt = f"""\
You are an assistant that has access to the following set of tools.
Here are the names and descriptions for each tool:

(rendered tools)

Given the user instructions, for each instruction do the following:
  - Return the name and input of the tool to use.
  - Return your response as a JSON blob with 'name' and 'arguments' keys.
  - The 'arguments' should be a dictionary, with keys corresponding
  to the argument names and the values corresponding to the requested values.
"""


## prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ('user', "{input}")
    ]
)

In [95]:
## give instructions
instructions = [
                {"input" : "What is 2.1 times 3.5"},
                {"input" : "What is 30 - 12"},
                {"input" : "What is the current weather in Zurich"},
                {"input" : "Tell me about Greenland and its capital"}
]

In [96]:
## now connect this to chatgpt using LCEL
from langchain_core.output_parsers import JsonOutputParser

## LCEL chain
chain = (prompt
           |
         chatgpt
           |
         JsonOutputParser()) ##output parser

In [97]:
## map responses
responses = chain.map().invoke(instructions)

In [98]:
## responses
responses

[{'name': 'Multiplication Tool', 'arguments': {'num1': 2.1, 'num2': 3.5}},
 {'name': 'Subtraction Tool', 'arguments': {'minuend': 30, 'subtrahend': 12}},
 {'name': 'Weather Lookup', 'arguments': {'location': 'Zurich'}},
 {'name': 'CountryInfo', 'arguments': {'country': 'Greenland'}}]

In [ ]:
## toolkit defined
toolkit = {
    "multiplication tool": multiply,
    "subtraction tool": subtract,  # Make sure the tool name matches what's in responses
    "weather lookup": get_weather,
    "countryinfo": search_web  # Make sure the tool name matches what's in responses
}

for tool_call in responses:
  selected_tool = toolkit[tool_call['name'].lower()]
  print(f"Calling tool: {tool_call['name']}")

  # Extract arguments or provide defaults if missing
  arguments = tool_call.get('arguments', {})  # Get arguments or an empty dict if missing
  if selected_tool == multiply:
      arguments = {'a': arguments.get('a', 0), 'b': arguments.get('b', 0)} # Provide defaults for 'a' and 'b'
  elif selected_tool == subtract:
      arguments = {'a': arguments.get('a', 0), 'b': arguments.get('b', 0)}  # Provide defaults for 'a' and 'b' if needed

  tool_output = selected_tool.invoke(arguments)
  print()